## Load, plot, edit Actiwatch AWD file

JWE 10Mar19

This python notebook is designed to help edit the 'on' and 'off' wrist segment markers in an actigraphy AWD file.  It is meant to be run cell by cell... You can try running all of them at once, nothing bad will happen, but maybe nothing good either.

The AWD file format is assumed to be a space separated text file and have a header with 7 rows. The two columns of data are assumed to be activity and marker (which is an M).

Any markers in the AWD file are matched up with transition points (TP) between segments of higher and lower activiy (counts over or below 40 over a weighted 5 minute window centered on the time point, respectively).  If a sleep log file (csv) exists then the markers are matched up with data in this file.  The sleep log file markers are assumed to be pairs and take timing precedence over the TPs.

If there are no markers in the file, never fear, low activity segments are used to establish guessed markers (z).  Do note that this tends to detect sleep period (not fully contiguously) or naps in addition to off watch periods.

The markers found in the AWD file, log file and guessed markers are output to an Mtimes csv file which is a listing of dates and times along with the marker code.  M for marker, z for guessed marker.  This Mtimes file can be edited at will and read back in.  Note that the programs assume a smart user who will list a pair of time points per segment i.e. an 'on' time and and 'off' time.

In order to help with segment identification there is also a rudimentary plotting program that will plot activity data along with marked segments or comments from a sleep log file if it exists.  Any number of marked segments can be displayed as long as they are identified with a different letter.  Activity data can be plotted as single or double day.  Plotting takes a long time (on the order of minutes).

Finally (or maybe this should have been mentioned first), data can be clipped to a subset of the recorded data before the markers are generated.  Note that currently it is assumed the log data are within the clipped limits.

In this notebook, the data is read and marker file and data file (with time stamps) are written out  Two plots are then made: te first plot is of the detected markers and log file data (if any), the second is the detected and guessed markers.  There is also a cell to read the marker file is also (assumed to be edited) and a new plot is made of the markers found in the file.

Disclaimer:  The marker identification function does a pretty good job but isn't perfect; especially given the fact that there's no way of telling whether low activity periods are off watch segments from the data itself. The marker file does let the user switch any guessed markers to actual markers, adjust the timing of the markers, or remove errant marked segments.  No complaints about marker detection will be entertained!  There may also be oddities with display, most of these are just display issues rather than marker issues at this point.

Lastly, it is possible to restrict the markers that are written out to the final data or marker file to a single type.


Import required libraries

In [ ]:
import os,sys,inspect
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir) 

import actigraPy.actigraPy as act
import importlib

Set up participant and file information.

### Edit this part

In [ ]:
# filname prefix (usally the subject inits)
sub = 'TEST' 
# data directory
dat_dir = '/Users/evansjw/ETPB/sleep/actiwatch/actigraPy/data/'
out_dir = '/Users/evansjw/ETPB/sleep/actiwatch/data/output'

fn = os.path.join(dat_dir, sub +'.AWD')
log_fn = os.path.join(dat_dir, sub + '_sleeplog.csv')
#lim = ['22-Feb-16 12:00 AM','12-Mar-16 11:59 PM']
lim=[]

### Just run the rest of the cells... 
The cell defaults can be tweaked but should run as is...

Read in the data, clip to dates if required, get markers and write markers out to file.

In [ ]:
importlib.reload(act)
# read the data
awd_dat = act.read_AWD(fn)

print('start: ',awd_dat['DateTime'][0])
print('stop: ',awd_dat['DateTime'][-1])

#clip_data
if lim:
    clip_dat = act.clip_dat(lim,awd_dat)
    fn_pref = os.path.join(out_dir, sub + '_clipped')
else:
    clip_dat = awd_dat
    fn_pref = os.path.join(out_dir,sub)

# despike
dat = act.despike(clip_dat['activity'],4,10)
clip_dat['dat'] = dat

# get markers
mk_idx,comments = act.get_markers(clip_dat,log_fn)

# write markers
if comments:
    act.write_Mtimes(clip_dat,mk_idx,fn_pref,comments)
else:
    act.write_Mtimes(clip_dat,mk_idx,fn_pref)
    
# write data
act.write_dat(clip_dat,mk_idx,fn_pref)

### Plotting


Plot the markers and log data (if there is any - need to add check) and saves to png.

In [ ]:
importlib.reload(act)
if comments:
    act.plot_awd(clip_dat,{'':[],'m':mk_idx['m'],'l':mk_idx['l']},show=False,fn_pref=os.path.join(out_dir,sub+'_M+log'),comments=comments,max_act=500)
else:
    print('no log data')

### Plot the guessed markers too

In [ ]:
importlib.reload(act)
try:
    act.plot_awd(clip_dat,{'m':mk_idx['m'],'l':mk_idx['l'],'z':mk_idx['z']},show=False,fn_pref=os.path.join(out_dir,sub+'_M+guess'),plot_type='single',debug=True)
except:
    act.plot_awd(clip_dat,{'m':mk_idx['m'],'':[],'z':mk_idx['z']},show=False,plot_type='single',fn_pref=os.path.join(out_dir,sub+'_M+guess'))
    
    

20-Feb-16
21-Feb-16
22-Feb-16
